In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [4]:
region = 'wuhan'
elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

In [5]:
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi)        
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi)
l58 = l8.merge(l5)
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}

year = 2020
startDate = str(year) + '-01-01'
endDate = str(year) + '-12-31'
refer_img = l58.filterDate(startDate, endDate).map(maskSR).median().clip(roi)
Map.addLayer(refer_img, visParams,'refer_img')

# PW和SW的面积（1999-2020）

## 多个，our result

In [ ]:
pwater = []
swater = []
mwater = []

# item = [['region A','daqing'],['region B','tianjin'],['region C','wuhan'],['region D','prd'],['region E','tibet']]
item = [['region A','daqing','(a1)'],['region B','tianjin','(b1)'],['region C','wuhan','(c1)'],['region D','prd','(d1)'],['region E','tibet','(e1)']]
for it in item:
    if it[0] == 'region A':
        re = [123.9355, 45.8362,125.3055, 46.8762]
    elif it[0] == 'region B':
        re = [115.9244, 38.6272,117.4944,39.6372]
    elif it[0] == 'region C':
        re = [113.7393, 29.8642,115.0993, 30.9242]
    elif it[0] == 'region D':
        re = [112.7614, 22.2347,114.0514, 23.2547]   
    elif it[0] == 'region E':
        re = [89.8826, 34.6579,91.3626, 35.8279]
    
    roi = ee.Geometry.Rectangle(re)
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    frequency = ee.Image('users/311605001111/WF/' + it[1] + '_WF_9920')
    permanent = frequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
    season= ee.ImageCollection([frequency.gte(0.25),frequency.lt(0.75)]).sum().eq(2).rename('waterclass')
    pw = ee.ImageCollection([permanent,season,basemap]).sum()
    
    pw_pw_area = pw.eq(2).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_pw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    print('PW: ',areas.get('waterclass').getInfo())
    pwater.append(areas.get('waterclass').getInfo())
    
    pw_sw_area = pw.eq(1).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_sw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    print('SW: ',areas.get('waterclass').getInfo())
    swater.append(areas.get('waterclass').getInfo())
    
    pw_max_area = pw.gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_max_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    print('MW: ',areas.get('waterclass').getInfo())
    mwater.append(areas.get('waterclass').getInfo())

In [ ]:
print(pwater)
print(swater)
print(mwater)

## 多个,jrc

In [ ]:
# occurrence9920 = ee.Image("users/311605001111/JRC_wuhan_9920").select(['waterclass'],['occurrence'])
# Map.addLayer(occurrence9920,{'palette':['white','green'],'min':0,'max':1}, 'occurrence9920')

# frequency = ee.Image("users/311605001111/JRC_wuhan_9920").select(['waterclass'],['occurrence'])
# permanent = frequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
# season= ee.ImageCollection([frequency.gte(0.25),frequency.lt(0.75)]).sum().eq(2).rename('waterclass')
# pw = ee.ImageCollection([permanent,season]).sum()
# Map.addLayer(pw,{'min': 0.0,'max': 2.0,'palette': ['ffffff', 'fffcb8', '0905ff']}, 'pw')

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee

# item = [['region A','daqing'],['region B','tianjin'],['region C','wuhan'],['region D','prd'],['region E','tibet']]
item = [['region A','daqing','(a1)'],['region B','tianjin','(b1)'],['region C','wuhan','(c1)'],['region D','prd','(d1)'],['region E','tibet','(e1)']]
for it in item:
    if it[0] == 'region A':
        re = [123.9355, 45.8362,125.3055, 46.8762]
    elif it[0] == 'region B':
        re = [115.9244, 38.6272,117.4944,39.6372]
    elif it[0] == 'region C':
        re = [113.7393, 29.8642,115.0993, 30.9242]
    elif it[0] == 'region D':
        re = [112.7614, 22.2347,114.0514, 23.2547]   
    elif it[0] == 'region E':
        re = [89.8826, 34.6579,91.3626, 35.8279]
    
    fig = plt.figure(figsize=(12, 8))
    vis = {'min': 0,'max': 2,'palette':['#FFFFFF','#87CEFA','#0000FF']}
    
    basemap = ee.Image.constant(0).clip(ee.Geometry.Rectangle(re)).rename('waterclass')
    frequency = ee.Image('users/311605001111/JRC/' + 'JRC_' + it[1] + '_9920')
    permanent = frequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
    season= ee.ImageCollection([frequency.gte(0.25),frequency.lt(0.75)]).sum().eq(2).rename('waterclass')
    pw = ee.ImageCollection([permanent,season]).sum().visualize(**vis)
    
    ax = cartoee.get_map(pw,region=re)
    cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
    # cartoee.add_colorbar(ax, vis,cmap = 'Blues',loc="right",label="water frequency", orientation="vertical")
    # ax.set_title(label = it[0] + ' (1999-2020)', fontsize=20)
    ax.set_title(label = it[2] + '     GSWD ' + '(' + it[0] + ' / 1999-2020)', fontsize=20)

In [8]:
pwater = []
swater = []
mwater = []

item = [['region A','daqing'],['region B','tianjin'],['region C','wuhan'],['region D','prd'],['region E','tibet']]

for it in item:
    if it[0] == 'region A':
        re = [123.9355, 45.8362,125.3055, 46.8762]
    elif it[0] == 'region B':
        re = [115.9244, 38.6272,117.4944,39.6372]
    elif it[0] == 'region C':
        re = [113.7393, 29.8642,115.0993, 30.9242]
    elif it[0] == 'region D':
        re = [112.7614, 22.2347,114.0514, 23.2547]   
    elif it[0] == 'region E':
        re = [89.8826, 34.6579,91.3626, 35.8279]
   
    roi = ee.Geometry.Rectangle(re)
    basemap = ee.Image.constant(0).clip(ee.Geometry.Rectangle(re)).rename('waterclass')
    frequency = ee.Image('users/311605001111/JRC/' + 'JRC_' + it[1] + '_9920')
    permanent = frequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
    season= ee.ImageCollection([frequency.gte(0.25),frequency.lt(0.75)]).sum().eq(2).rename('waterclass')
    pw = ee.ImageCollection([permanent,season]).sum()
    
    pw_pw_area = pw.eq(2).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_pw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    print('PW: ',areas.get('waterclass').getInfo())
    pwater.append(areas.get('waterclass').getInfo())
    
    pw_sw_area = pw.eq(1).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_sw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    print('SW: ',areas.get('waterclass').getInfo())
    swater.append(areas.get('waterclass').getInfo())
    
    pw_max_area = pw.gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_max_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    print('MW: ',areas.get('waterclass').getInfo())
    mwater.append(areas.get('waterclass').getInfo())

PW:  984.9442066775243
SW:  548.6490503931666
MW:  1533.5932570707064
PW:  166.46215825629653
SW:  764.8019590304176
MW:  931.264117286713
PW:  1550.7322660466464
SW:  1714.0019471622788
MW:  3264.7342132091653
PW:  2466.975588165705
SW:  1043.1979278209185
MW:  3510.1735159868094
PW:  2137.720719789316
SW:  267.7133821797643
MW:  2405.4341019690855


In [ ]:
print(pwater)
print(swater)
print(mwater)

##  制图

In [ ]:
# our result
our_pw = [1073.6583851042938, 231.29111243872381, 1664.908380786999, 2457.1143701154424, 2101.613526165305]
our_sw = [700.0475015721559, 636.0955432330977, 1370.5313876992209, 796.8368395648121, 322.9033363355685]
our_mw = [1773.7058866764949, 867.3866556717976, 3035.4397684865316, 3253.9512096804483, 2424.5168625007195]

# jrc
# jrc_pw = [993.9115928846336, 160.49209321191506, 1478.898685273799, 2439.0017645175226, 2148.8082088924098]
# jrc_sw = [690.2540363991794, 653.8117290643022, 1197.6200670173885, 748.7212888136327, 318.49179606343296]
# jrc_mw = [1684.1656292838272, 814.3038222762314, 2676.5187522913843, 3187.7230533313846, 2467.3000049557513]

jrc_pw = [984.9442066775242, 166.46215825629656, 1550.7322660466466, 2466.9755881657047, 2137.7207197893163]
jrc_sw = [548.6490503931665, 764.8019590304176, 1714.0019471622793, 1043.1979278209185, 267.7133821797643]
jrc_mw = [1533.5932570707064, 931.264117286713, 3264.7342132091653, 3510.1735159868094, 2405.434101969085]

### 一张图

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

plt.figure(figsize=(8,6))
year = np.arange(1,6,1)

plt.plot(year, our_pw,'*-b' ,label = 'Our result--PW')
plt.plot(year, jrc_pw,'*-r' ,label = 'GSWD--PW')

plt.plot(year, our_sw,'x--b',label = 'Our result--SW')
plt.plot(year, jrc_sw,'x--r',label = 'GSWD--SW')

plt.plot(year, our_mw,'^:b',label = 'Our result--MW')
plt.plot(year, jrc_mw,'^:r',label = 'GSWD--MW')
# plt.plot(year, our_wo,'o-.m' ,label = 'Our result--SW')



# plt.title("Area change in various water (region C / 2000-2020)", fontsize=18)
plt.title("Water area in each areas (1999-2020)", fontsize=18)
plt.xlabel("Region", fontsize=18)
plt.ylabel('Area/km²', fontsize=18)

plt.ylim(0,4000)
plt.tick_params(labelsize=15)
plt.xticks([1,2,3,4,5],['region A','region B','region C','region D','region E'])
plt.legend(prop={'size':12},loc='upper left')
plt.show()

### 多张图

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

plt.figure(figsize=(8,6))
year = np.arange(1,6,1)

plt.plot(year, our_pw,'*-b' ,label = 'Our result')
plt.plot(year, jrc_pw,'^--r' ,label = 'GSWD')



# plt.title("Area change in various water (region C / 2000-2020)", fontsize=18)
plt.title("Area of permanent water in each areas (1999-2020)", fontsize=18)
plt.xlabel("Region", fontsize=18)
plt.ylabel('Area/km²', fontsize=18)

plt.ylim(0,4000)
plt.tick_params(labelsize=15)
plt.xticks([1,2,3,4,5],['region A','region B','region C','region D','region E'])
plt.legend(prop={'size':12},loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

plt.figure(figsize=(8,6))
year = np.arange(1,6,1)

plt.plot(year, our_sw,'*-b',label = 'Our result')
plt.plot(year, jrc_sw,'^--r',label = 'GSWD')



# plt.title("Area change in various water (region C / 2000-2020)", fontsize=18)
plt.title("Area of seasonal water in each areas (1999-2020)", fontsize=18)
plt.xlabel("Region", fontsize=18)
plt.ylabel('Area/km²', fontsize=18)

plt.ylim(0,4000)
plt.tick_params(labelsize=15)
plt.xticks([1,2,3,4,5],['region A','region B','region C','region D','region E'])
plt.legend(prop={'size':12},loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

plt.figure(figsize=(8,6))
year = np.arange(1,6,1)

plt.plot(year, our_mw,'*-b',label = 'Our result')
plt.plot(year, jrc_mw,'^--r',label = 'GSWD')


# plt.title("Area change in various water (region C / 2000-2020)", fontsize=18)
plt.title("Area of maximum water in each areas (1999-2020)", fontsize=18)
plt.xlabel("Region", fontsize=18)
plt.ylabel('Area/km²', fontsize=18)

plt.ylim(0,4000)
plt.tick_params(labelsize=15)
plt.xticks([1,2,3,4,5],['region A','region B','region C','region D','region E'])
plt.legend(prop={'size':12},loc='upper left')
plt.show()

## 单个

In [ ]:
# 实验结果
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
result = ee.Image('users/311605001111/WF/' + region + '_WF_' + '9920').rename('waterclass')
permanent = result.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
season= ee.ImageCollection([result.gte(0.25),result.lt(0.75)]).sum().eq(2).rename('waterclass')
pw = ee.ImageCollection([permanent,season,basemap]).sum()
Map.addLayer(pw.selfMask(),{'palette':['#87CEFA', 'blue'],'min':1,'max':2},"pw")

pw_area = pw.eq(2).multiply(ee.Image.pixelArea()).divide(1e6)
areas = pw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print('PW: ',areas.get('waterclass').getInfo())

pw_no_area = pw.eq(1).multiply(ee.Image.pixelArea()).divide(1e6)
areas = pw_no_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print('SW: ',areas.get('waterclass').getInfo())

In [ ]:
# JRC

frequency = ee.Image('users/311605001111/' + 'JRC_' + region + '_9920')
# basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
# dataset = ee.ImageCollection('JRC/GSW1_3/MonthlyHistory').filterDate('1999-01-01', '2020-12-31')
# water_pixels = dataset.map(lambda i : i.select('water').clip(roi).gt(1)).sum()
# valid_pixels = dataset.map(lambda i : i.select('water').clip(roi).gt(0)).sum()
# frequency = water_pixels.divide(valid_pixels).rename('waterclass')
# Map.addLayer(frequency.selfMask(),{'palette':['white', 'blue'],'min':0,'max':1},"frequency")

permanent = frequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
season= ee.ImageCollection([frequency.gte(0.25),frequency.lt(0.75)]).sum().eq(2).rename('waterclass')
jrc_waterclass = ee.ImageCollection([permanent,season,basemap]).sum()
Map.addLayer(jrc_waterclass.selfMask(),{'palette':['#bfd69b', 'green'],'min':1,'max':2},"jrc")

pw_area = jrc_waterclass.eq(2).multiply(ee.Image.pixelArea()).divide(1e6)
areas = pw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print('PW: ',areas.get('waterclass').getInfo())

pw_no_area = jrc_waterclass.eq(1).multiply(ee.Image.pixelArea()).divide(1e6)
areas = pw_no_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print('SW: ',areas.get('waterclass').getInfo())

In [ ]:
# 实验结果（未校正）
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
result = ee.Image('users/311605001111/WF_nocorrect/' + region + '_WF_' + '9920').rename('waterclass')
permanent = result.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
season= ee.ImageCollection([result.gte(0.25),result.lt(0.75)]).sum().eq(2).rename('waterclass')
pw_no = ee.ImageCollection([permanent,season,basemap]).sum()
Map.addLayer(pw_no.selfMask(),{'palette':['purple']},"pw_no")

pw_area = pw_no.eq(2).multiply(ee.Image.pixelArea()).divide(1e6)
areas = pw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print('PW: ',areas.get('waterclass').getInfo())

pw_no_area = pw_no.eq(1).multiply(ee.Image.pixelArea()).divide(1e6)
areas = pw_no_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print('SW: ',areas.get('waterclass').getInfo())

In [ ]:
# Maryland

basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
Maryland = ee.Image("users/311605001111/MARYLAND_99-20").clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').eq(2).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= Maryland.select('waterclass').eq(7).rename('waterclass')
maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
Map.addLayer(maryland_waterclass.selfMask(),{'palette':['#4ad6c3','red'],'min':1,'max':2},"maryland")

pw_area = maryland_waterclass.eq(2).multiply(ee.Image.pixelArea()).divide(1e6)
areas = pw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print('PW: ',areas.get('waterclass').getInfo())

pw_no_area = maryland_waterclass.eq(1).multiply(ee.Image.pixelArea()).divide(1e6)
areas = pw_no_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
print('SW: ',areas.get('waterclass').getInfo())

# PW和SW的面积（2000，2005，2010，2015，2020）

In [ ]:
# # 大庆市
# region = 'daqing'
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# region = 'tianjin'
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
region = 'wuhan'
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# region = 'prd'
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# region = 'tibet'
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

## JRC

In [ ]:
pwater = []
swater = []
for year in range(2000,2021,5):
    print(year)
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    
    JRC = ee.Image('JRC/GSW1_3/YearlyHistory/' + str(year)).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
    pw = ee.ImageCollection([JRC,basemap]).sum()
    # Map.addLayer(pw.selfMask(),{'palette':['#87CEFA', 'blue'],'min':1,'max':2},"pw")
    
    pw_area = pw.eq(2).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    print('PW: ',areas.get('waterclass').getInfo())
    pwater.append(areas.get('waterclass').getInfo())
    
    pw_no_area = pw.eq(1).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_no_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    print('SW: ',areas.get('waterclass').getInfo())
    swater.append(areas.get('waterclass').getInfo())

In [ ]:
print(pwater)
print(swater)

## Maryland

In [ ]:
pwater = []
swater = []
for year in range(2000,2021,5):
    print(year)
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    
    Maryland = ee.Image('users/311605001111/Maryland/Maryland_nationwide_' + str(year)).clip(roi).select('b1').rename('waterclass')
    Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
    Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
    pw = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()    
    # Map.addLayer(pw.selfMask(),{'palette':['#87CEFA', 'blue'],'min':1,'max':2},"pw")
    
    pw_area = pw.eq(2).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    print('PW: ',areas.get('waterclass').getInfo())
    pwater.append(areas.get('waterclass').getInfo())
    
    pw_no_area = pw.eq(1).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_no_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    print('SW: ',areas.get('waterclass').getInfo())
    swater.append(areas.get('waterclass').getInfo())

In [ ]:
print(pwater)
print(swater)

## Our result

In [ ]:
# 计算
pwater = []
swater = []
for year in range(2000,2021,5):
    print(year)
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    result = ee.Image('users/311605001111/WF/' + region + '_WF_' + str(year)).rename('waterclass')
    permanent = result.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
    season= ee.ImageCollection([result.gte(0.25),result.lt(0.75)]).sum().eq(2).rename('waterclass')
    pw = ee.ImageCollection([permanent,season,basemap]).sum()
    # Map.addLayer(pw.selfMask(),{'palette':['#87CEFA', 'blue'],'min':1,'max':2},"pw")
    
    pw_area = pw.eq(2).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    print('PW: ',areas.get('waterclass').getInfo())
    pwater.append(areas.get('waterclass').getInfo())
    
    pw_no_area = pw.eq(1).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = pw_no_area.reduceRegion(**{'reducer': ee.Reducer.sum(),'geometry': roi,'scale': 30,'maxPixels': 1e14 })
    print('SW: ',areas.get('waterclass').getInfo())
    swater.append(areas.get('waterclass').getInfo())

In [ ]:
print(pwater)
print(swater)

In [ ]:
# region A
# pwater = [1294.7238098718306, 1162.2147299209773, 1175.6265415699902, 1425.1934993771754, 1522.2998796074125]
# swater = [529.3891544596097, 614.8230854039382, 505.9648056078811, 576.7378436699851, 759.116174734088]

# # region B
# pwater = [389.8300949300209, 499.66594719242994, 359.6137283771112, 367.38701840706244, 483.3019181285885]
# swater = [307.02835881440876, 322.3283737304871, 399.6433022098337, 381.20521895196885, 446.7491821932476]

# region C
pwater = [1742.803061500307, 1767.3411693457658, 2014.336046780104, 1881.047325368986, 1967.926536578787]
swater = [1200.29945551262, 1264.7137798028177, 1171.6298557849873, 960.8201012417004, 1135.8482452760952]

# # region D
# pwater = [2736.3767901270767, 2743.354059797078, 2616.636802060956, 2445.557540189891, 2353.0231161996153]
# swater = [655.1116738118164, 689.4101057420261, 542.3113952600945, 453.368261613696, 434.7849028281628]

# # region E
# pwater = [1795.6140424029343, 2032.8778920203717, 2159.4533749978427, 2287.5011316205296, 2572.575699173833]
# swater = [86.32433359075402, 113.03516058995584, 128.6198355527511, 138.5201381679648, 85.35564178690635]

## 制图

In [ ]:
# JRC
# pwater = [1197.703182934414, 1218.5285708682795, 1374.655082180913, 1410.6191766351205, 1450.0363113039618]
# swater = [1699.645732732901, 1779.9593796751194, 1642.1531701698925, 1397.7903686379516, 1816.1106822093943]

# # Maryland
# pwater = [2369.184113423742, 2032.2425352816872, 2596.2345063417843, 2126.259792251638, 2200.3939879603254]
# swater = [1310.530862357164, 1463.0638249783008, 1434.584744375592, 1264.094984787088, 1536.293257519056]

# # our result
pwater = [1742.803061500307, 1767.3411693457658, 2014.336046780104, 1881.047325368986, 1967.926536578787]
swater = [1200.29945551262, 1264.7137798028177, 1171.6298557849873, 960.8201012417004, 1135.8482452760952]

In [ ]:
jrc = np.array(pwater) + np.array(swater)
print(jrc)

In [ ]:
jrc = [2897.34891567, 2998.48795054, 3016.80825235, 2808.40954527, 3266.14699351]
maryland = [3679.71497578, 3495.30636026, 4030.81925072, 3390.35477704, 3736.68724548]
our = [2943.10251701, 3032.05494915, 3185.96590257, 2841.86742661, 3103.77478185]

In [ ]:
# jrc = [1197.703182934414, 1218.5285708682795, 1374.655082180913, 1410.6191766351205, 1450.0363113039618]
# maryland = [2369.184113423742, 2032.2425352816872, 2596.2345063417843, 2126.259792251638, 2200.3939879603254]
# our = [1742.803061500307, 1767.3411693457658, 2014.336046780104, 1881.047325368986, 1967.926536578787]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

plt.figure(figsize=(8,6))
year = np.arange(2000,2021,5)
plt.plot(year, our,'*-b' ,label = 'our result')
plt.plot(year, jrc,'^--r',label = 'GSWD')
plt.plot(year, maryland,'2:g',label = 'GLAD')
# plt.title("Area change in various water (region C / 2000-2020)", fontsize=18)
plt.title("maximum water area (region C / 2000-2020)", fontsize=18)
plt.xlabel("Year", fontsize=18)
plt.ylabel('Area/km²', fontsize=18)

# year = np.arange(2000,2021,1).reshape(-1,1)
# model = LinearRegression()  # 创建模型
# model.fit(year, pw)  # 拟合
# x = [[2000],[2005],[2010],[2015],[2020]]
# y = model.predict(x)
# plt.plot(x, y, 'g-',label = 'Fitting trend')
plt.ylim(0,4200)
plt.tick_params(labelsize=15)
plt.xticks([2000,2005,2010,2015,2020])
plt.legend(prop={'size':15})
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

plt.figure(figsize=(8,6))
year = np.arange(2000,2021,5)
plt.plot(year, pwater,'*-b' ,label = 'Permanent water')
plt.plot(year, swater,'^--r',label = 'Seasonal water')
# plt.title("Area change in various water (region C / 2000-2020)", fontsize=18)
plt.title("GSWD (region C / 2000-2020)", fontsize=18)
plt.xlabel("Year", fontsize=18)
plt.ylabel('Area/km²', fontsize=18)

# year = np.arange(2000,2021,1).reshape(-1,1)
# model = LinearRegression()  # 创建模型
# model.fit(year, pw)  # 拟合
# x = [[2000],[2005],[2010],[2015],[2020]]
# y = model.predict(x)
# plt.plot(x, y, 'g-',label = 'Fitting trend')
plt.ylim(0,3000)
plt.tick_params(labelsize=15)
plt.xticks([2000,2005,2010,2015,2020])
plt.legend(prop={'size':15})
plt.show()